In [ ]:
"""
Created on Fri Nov 24 15:28 2023

SMB formatting from Christoph

Author: @claraburgard

"""

In [ ]:
import xarray as xr
import pandas as pd
#import matplotlib.pyplot as plt
#from matplotlib.font_manager import FontProperties
#import matplotlib.gridspec as gridspec
import numpy as np
import cftime
#import seaborn as sns

In [ ]:
def SMBcomponents_to_gt(SMB_array, nisf,mask,ice_frac,data_start=1980, data_end=2100):
    """ This function returns a daily time series of absolute SMB component values, expressed as gigatons. 35 000 m = MAR resolution 1e12 factor to convert kg/m² in Gt """
    
    ice=ice_frac.where(mask.values==nisf)
    reso=4.*1000.
    data = SMB_array*ice.values#*area.values
    # Make sure only wanted time frame is used
    #data = data.loc[str(data_start) + ‘-01-01’:str(data_end) + ‘-12-31’]
    # Convert to gigatons and sum up spatially over the AIS
    sum_spatial = data.sum(dim=['x', 'y']) * ((reso * reso) / (1e12))
    return sum_spatial

In [ ]:
# =============================================================================
# # OPEN the ICE MASK
# =============================================================================
mask = xr.open_dataset('BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc'
            ,decode_times=False)

isfarea=mask["isf_area_rignot"]
names=mask["isf_name"]
nisf_values=mask['Nisf'].values
mask=mask['ISF_mask']

ice=xr.open_dataset("BedMachine_v2_aggregated4km_allvars_cut.nc")
ice=ice["isf_conc"]
forcing='ACCESS1.3'
yearstart=1980
yearend=2100
ssp='rcp8.5'

In [ ]:
# =============================================================================
# # Create the future xarray dataset
# =============================================================================
# Generate nisf and time dimensions
# Generate nisf and time dimensions
#nisf_values = np.arange(0, 64, 1) # From 0 to 216
time_values = np.arange(yearstart, yearend+1, 1) # From 1980 to 2100 #will be converted later with a real time
# Generate random data for the dataset
data1 = np.random.rand(len(nisf_values), len(time_values))*0
data2 = np.random.rand(len(nisf_values), len(time_values))*0
dates = xr.cftime_range(start=str(yearstart)+"-01-01", end=str(yearend)+"-12-31", freq='AS')
                        

In [ ]:
# =============================================================================
# # OPEN the SMB file
# =============================================================================
var_list=[“SMB”,“RU”,“SF”,“RF”,“ME”]
var_list=[“SMB”]
for var in var_list:
    MAR=xr.open_dataset("in/MAR3.11-"+forcing+"_"+ssp+"-onbedmachinev2-4km_"+str(yearstart)+"-"+str(yearend)+".nc",decode_times=False)
    MAR["time"]=dates
    # Create the xarray dataset
    dataset = xr.Dataset(
    data_vars={var: (['Nisf', 'time'], data1),
          var+'org': (['Nisf', 'time'], data2)},
    coords={‘Nisf’: nisf_values, 'time': dates})
    
    varint=MAR[var]
    varorg=MAR[var+'org']
    area=MAR["AREA"]
    newice=ice*area.values
    MAR["time"]=dates
    del(MAR)
    for nisf in range(0,len(nisf_values)):
        print (nisf,nisf_values[nisf], names.values[nisf])
        if np.isfinite(isfarea[nisf].values):
            print (nisf, "ok")
            dataset[var][nisf]=SMBcomponents_to_gt(varint, nisf_values[nisf],mask,newice).values
            dataset[var+‘org’][nisf]=SMBcomponents_to_gt(varorg, nisf_values[nisf],mask,newice).values

    # Save the xarray dataset to a NetCDF file
    historical=dataset.sel(time=slice("1980-01-01","2014-12-31"))
    if forcing != "ERA5":
        ssp585=dataset.sel(time=slice("2015-01-01",str(yearend)+"-12-31"))
        output_filename = "./out/"+forcing+'_'+var+'_'+ssp+'-2015-'+str(yearend)+'.nc'
        ssp585.to_netcdf(output_filename)
        print(“Xarray dataset saved to “+output_filename)
    
    output_filename = "./out/"+forcing+'_'+var+‘_historical-‘+str(yearstart)+‘-2014.nc’
    historical.to_netcdf(output_filename)
    print(“Xarray dataset saved to “+output_filename)